# SQL Functions

## Spark Session

In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .master("local")
         .appName("Spark session")
         .getOrCreate())

## Read Data Frame

In [2]:
df = (spark
      .read
      .parquet('data/part-r-00000-1a9822ba-b8fb-4d8e-844a-ea30d0801b9e.gz.parquet'))

df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



## Import all functions

In [6]:
from pyspark.sql.functions import *



In [22]:
# select and sort some data
df_sel = df.select('ORIGIN_COUNTRY_NAME', 'count').distinct().sort(asc('ORIGIN_COUNTRY_NAME'))
df_sel.show(5)

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|        Afghanistan|    2|
|            Algeria|    1|
|             Angola|   18|
|           Anguilla|   20|
|Antigua and Barbuda|  121|
+-------------------+-----+
only showing top 5 rows



## Aggregate Functions

In [23]:
# first
df_sel.agg(first('ORIGIN_COUNTRY_NAME')).show()

+---------------------------------+
|first(ORIGIN_COUNTRY_NAME, false)|
+---------------------------------+
|                      Afghanistan|
+---------------------------------+



In [24]:
# last
df_sel.agg(last('ORIGIN_COUNTRY_NAME')).show()

+--------------------------------+
|last(ORIGIN_COUNTRY_NAME, false)|
+--------------------------------+
|                         Vietnam|
+--------------------------------+



In [25]:
# min 
df_sel.agg(min('count')).show()

+----------+
|min(count)|
+----------+
|         1|
+----------+



In [26]:
# max
df_sel.agg(max('count')).show()

+----------+
|max(count)|
+----------+
|    348113|
+----------+



## Collection Functions

In [32]:
# contain
df_sel[df_sel['ORIGIN_COUNTRY_NAME'].like('%ussia%')].show()

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|             Russia|  156|
+-------------------+-----+



In [40]:
# explode
#
# Input:
# FieldA    FieldB    ArrayField
# 1         A         {1,2,3}
# 2         B         {3,5}
# 
# Result:
# FieldA    FieldB    ExplodedField
# 1         A         1
# 1         A         2
# 1         A         3
# 2         B         3
# 2         B         5

df_explode = spark.createDataFrame([(1, "A", [1,2,3]), (2, "B", [3,5])],["col1", "col2", "col3"])
df_explode.show()

+----+----+---------+
|col1|col2|     col3|
+----+----+---------+
|   1|   A|[1, 2, 3]|
|   2|   B|   [3, 5]|
+----+----+---------+



In [41]:
df_explode.withColumn("col3", explode(df_explode.col3)).show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   A|   1|
|   1|   A|   2|
|   1|   A|   3|
|   2|   B|   3|
|   2|   B|   5|
+----+----+----+



In [63]:
# flatten
# create single array from array of arrays 
df_flatten = spark.createDataFrame([([[1, 2, 3], [4, 5], [6]],), ([None, [4, 5]],)], ['data'])
df_flatten.show()

+--------------------+
|                data|
+--------------------+
|[[1, 2, 3], [4, 5...|
|          [, [4, 5]]|
+--------------------+



In [66]:
df_flatten.select(flatten(df_flatten.data)).show()

+------------------+
|     flatten(data)|
+------------------+
|[1, 2, 3, 4, 5, 6]|
|              null|
+------------------+



## Date time functions

In [98]:
from pyspark.sql import Row 

# yyyy-MM-dd or yyyy-MM-dd HH:mm:ss.SSSS format
dtimes = spark.sparkContext.parallelize([Row(input_date='2019-01-23', input_timestamp='2019-01-23 01:23:45.6789')]).toDF()
dtimes.show()

+----------+--------------------+
|input_date|     input_timestamp|
+----------+--------------------+
|2019-01-23|2019-01-23 01:23:...|
+----------+--------------------+



In [99]:
# current date
dtimes.select(current_date()).show()

+--------------+
|current_date()|
+--------------+
|    2019-10-03|
+--------------+



In [128]:
# date format
dtimes.select('input_date',
              date_format('input_date', 'yyyy/MM/dd'),
              date_format('input_timestamp', 'yyyy/MM/dd HH:mm:ss.SSSS')).printSchema()

root
 |-- input_date: string (nullable = true)
 |-- date_format(input_date, yyyy/MM/dd): string (nullable = true)
 |-- date_format(input_timestamp, yyyy/MM/dd HH:mm:ss.SSSS): string (nullable = true)



In [102]:
# date format
dtimes.select('input_date',
              date_format('input_date', 'yyyy/MM/dd'),
              date_format('input_timestamp', 'yyyy/MM/dd HH:mm:ss.SSSS')).show()

+----------+-----------------------------------+------------------------------------------------------+
|input_date|date_format(input_date, yyyy/MM/dd)|date_format(input_timestamp, yyyy/MM/dd HH:mm:ss.SSSS)|
+----------+-----------------------------------+------------------------------------------------------+
|2019-01-23|                         2019/01/23|                                  2019/01/23 01:23:...|
+----------+-----------------------------------+------------------------------------------------------+



In [137]:
# to_date, to_timestamp
(dtimes
 .select('input_date',
         'input_timestamp',
         to_date('input_date').alias('date'),
         to_timestamp('input_timestamp').alias('timestamp'),
        )
 .printSchema())

root
 |-- input_date: string (nullable = true)
 |-- input_timestamp: string (nullable = true)
 |-- date: date (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [138]:
(dtimes
 .select('input_date',
         'input_timestamp',
         to_date('input_date').alias('date'),
         to_timestamp('input_timestamp').alias('timestamp'),
        )
 .show())

+----------+--------------------+----------+--------------------+
|input_date|     input_timestamp|      date|           timestamp|
+----------+--------------------+----------+--------------------+
|2019-01-23|2019-01-23 01:23:...|2019-01-23|2019-01-23 01:23:...|
+----------+--------------------+----------+--------------------+



In [135]:
# string -> date -> string, string -> timestamp -> string, 
(dtimes
 .select('input_date',
         'input_timestamp',
         to_date('input_date').alias('date'),
         to_timestamp('input_timestamp').alias('timestamp'),
        )
 .select('input_date',
        'input_timestamp',
        'date',
        'timestamp',
        date_format('date', 'yyyy/MM/dd').alias('date_string'),
        date_format('timestamp', 'yyyy/MM/dd HH:mm:ss.SSSS').alias('timestamp_string'))
 .printSchema())

root
 |-- input_date: string (nullable = true)
 |-- input_timestamp: string (nullable = true)
 |-- date: date (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- date_string: string (nullable = true)
 |-- timestamp_string: string (nullable = true)



In [136]:
(dtimes
 .select('input_date',
         'input_timestamp',
         to_date('input_date').alias('date'),
         to_timestamp('input_timestamp').alias('timestamp'),
        )
 .select('input_date',
        'input_timestamp',
        'date',
        'timestamp',
        date_format('date', 'yyyy/MM/dd').alias('date_string'),
        date_format('timestamp', 'yyyy/MM/dd HH:mm:ss.SSSS').alias('timestamp_string'))
 .show())

+----------+--------------------+----------+--------------------+-----------+--------------------+
|input_date|     input_timestamp|      date|           timestamp|date_string|    timestamp_string|
+----------+--------------------+----------+--------------------+-----------+--------------------+
|2019-01-23|2019-01-23 01:23:...|2019-01-23|2019-01-23 01:23:...| 2019/01/23|2019/01/23 01:23:...|
+----------+--------------------+----------+--------------------+-----------+--------------------+



## Math Functions

In [155]:
from pyspark.sql import Row 
from math import pi

dmath = spark.sparkContext.parallelize([Row(input='1'),
                                       Row(input='0'),
                                       Row(input=pi),
                                       Row(input=None),
                                       ]).toDF()
dmath.show()

+-----------------+
|            input|
+-----------------+
|                1|
|                0|
|3.141592653589793|
|             null|
+-----------------+



In [156]:
# cos
dmath.select('input',
           cos(col('input'))).show()

+-----------------+------------------+
|            input|        COS(input)|
+-----------------+------------------+
|                1|0.5403023058681398|
|                0|               1.0|
|3.141592653589793|              -1.0|
|             null|              null|
+-----------------+------------------+



In [157]:
# floor
dmath.select('input',
           floor(col('input'))).show()

+-----------------+------------+
|            input|FLOOR(input)|
+-----------------+------------+
|                1|           1|
|                0|           0|
|3.141592653589793|           3|
|             null|        null|
+-----------------+------------+



In [158]:
# log
dmath.select('input',
           log(col('input'))).show()

+-----------------+------------------+
|            input|        LOG(input)|
+-----------------+------------------+
|                1|               0.0|
|                0|              null|
|3.141592653589793|1.1447298858494002|
|             null|              null|
+-----------------+------------------+



## Misc Functions (hashes)

In [166]:
# crc32
spark.createDataFrame([('ABC',)], ['a']).select(crc32('a').alias('crc32')).collect()

[Row(crc32=2743272264)]

In [168]:
# md5
spark.createDataFrame([('ABC',)], ['a']).select(md5('a').alias('md5')).collect()

[Row(md5='902fbdd2b1df0c4f70b4a5d23525e932')]

In [169]:
# sha1
spark.createDataFrame([('ABC',)], ['a']).select(sha1('a').alias('sha1')).collect()

[Row(sha1='3c01bdbb26f358bab27f267924aa2c9a03fcfdb8')]

In [170]:
# sha2
spark.createDataFrame([('ABC',)], ['a']).select(sha1('a').alias('sha2')).collect()

[Row(sha2='3c01bdbb26f358bab27f267924aa2c9a03fcfdb8')]

## Non-Aggregate Functions

In [173]:
df = spark.createDataFrame([('ABC', float('nan')), 
                            (None, 1.0)], 
                           ("a", "b"))
df.show()

+----+---+
|   a|  b|
+----+---+
| ABC|NaN|
|null|1.0|
+----+---+



In [174]:
# isnan
df.select(isnan('a'), isnan('b')).show()

+--------+--------+
|isnan(a)|isnan(b)|
+--------+--------+
|   false|    true|
|   false|   false|
+--------+--------+



In [177]:
# isnull
df.select(isnull('a'), isnull('b')).show()

+-----------+-----------+
|(a IS NULL)|(b IS NULL)|
+-----------+-----------+
|      false|      false|
|       true|      false|
+-----------+-----------+



In [182]:
df.where(isnull('a')).show()

+----+---+
|   a|  b|
+----+---+
|null|1.0|
+----+---+



In [180]:
df.where(col('a').isNull()).show()

+----+---+
|   a|  b|
+----+---+
|null|1.0|
+----+---+



In [204]:
df.where(col('a') == col('a')).show() # :)

+---+---+
|  a|  b|
+---+---+
|ABC|NaN|
+---+---+



In [185]:
df.where(~isnull('a')).show() # not

+---+---+
|  a|  b|
+---+---+
|ABC|NaN|
+---+---+



In [181]:
df.where(col('a').isNotNull()).show()

+---+---+
|  a|  b|
+---+---+
|ABC|NaN|
+---+---+



In [195]:
df.where(col('b') != 'NaN').show()

+----+---+
|   a|  b|
+----+---+
|null|1.0|
+----+---+



In [197]:
df.where(isnan('b')).show()

+---+---+
|  a|  b|
+---+---+
|ABC|NaN|
+---+---+



In [199]:
df.where(~isnan('b')).show()

+----+---+
|   a|  b|
+----+---+
|null|1.0|
+----+---+



## Sorting

In [222]:
dict = [
    {'value': 2}, 
    {'value': 3},
    {'value': None},  
    {'value': 1},
    {'value': 5},
]

df = spark.createDataFrame(dict)

df.show()

+-----+
|value|
+-----+
|    2|
|    3|
| null|
|    1|
|    5|
+-----+



### Sort

In [223]:
# asc
df.sort(asc('value')).show()

+-----+
|value|
+-----+
| null|
|    1|
|    2|
|    3|
|    5|
+-----+



In [224]:
# desc
df.sort(desc('value')).show()

+-----+
|value|
+-----+
|    5|
|    3|
|    2|
|    1|
| null|
+-----+



### Order By

In [227]:
# orderBy is alias for sort function
df.orderBy(asc('value')).show()

+-----+
|value|
+-----+
| null|
|    1|
|    2|
|    3|
|    5|
+-----+



In [226]:
df.orderBy(desc('value')).show()

+-----+
|value|
+-----+
|    5|
|    3|
|    2|
|    1|
| null|
+-----+



In [228]:
df.orderBy(col('value')).show()

+-----+
|value|
+-----+
| null|
|    1|
|    2|
|    3|
|    5|
+-----+



In [229]:
df.orderBy(col('value'), ascending=False).show()

+-----+
|value|
+-----+
|    5|
|    3|
|    2|
|    1|
| null|
+-----+



## String functions

In [264]:
dic = [
    {'name': 'Alice',
     'age': 35},
    {'name': 'Bob',
     'age': 23},
    {'name': 'Cindy',
     'age': 46},
      ]

df = spark.createDataFrame(dic)
df.show()

+---+-----+
|age| name|
+---+-----+
| 35|Alice|
| 23|  Bob|
| 46|Cindy|
+---+-----+



In [249]:
# match
df.filter(df.name.like('%ob%')).show()

+---+----+
|age|name|
+---+----+
| 23| Bob|
+---+----+



In [252]:
# in range
df.filter(df.name.isin('Bob', 'Cindy')).show()

+---+-----+
|age| name|
+---+-----+
| 23|  Bob|
| 46|Cindy|
+---+-----+



In [258]:
# concat
df.withColumn('res', concat(df.name, df.age, lit('yo'))).show()

+---+-----+---------+
|age| name|      res|
+---+-----+---------+
| 35|Alice|Alice35yo|
| 23|  Bob|  Bob23yo|
| 46|Cindy|Cindy46yo|
+---+-----+---------+



### Trim

In [265]:
df = df.withColumn('for_trim', concat(lit('  '), df.name, lit('  ')))

df.show()

+---+-----+---------+
|age| name| for_trim|
+---+-----+---------+
| 35|Alice|  Alice  |
| 23|  Bob|    Bob  |
| 46|Cindy|  Cindy  |
+---+-----+---------+



In [272]:
# left trim
df.select(ltrim(df.for_trim).alias('r')).show()

+-------+
|      r|
+-------+
|Alice  |
|  Bob  |
|Cindy  |
+-------+



In [273]:
# right trim
df.select(rtrim(df.for_trim).alias('r')).show()

+-------+
|      r|
+-------+
|  Alice|
|    Bob|
|  Cindy|
+-------+



In [274]:
# trim
df.select(trim(df.for_trim).alias('r')).show()

+-----+
|    r|
+-----+
|Alice|
|  Bob|
|Cindy|
+-----+



In [279]:
df = df.drop('for_trim')

### Regex

In [280]:
# select
df.filter(df['name'].rlike('.ob$')).show()

+---+----+
|age|name|
+---+----+
| 23| Bob|
+---+----+



In [284]:
# replace
df.withColumn('res', regexp_replace('name', 'ob', 'OB')).show()

+---+-----+-----+
|age| name|  res|
+---+-----+-----+
| 35|Alice|Alice|
| 23|  Bob|  BOB|
| 46|Cindy|Cindy|
+---+-----+-----+



In [296]:
# regex extract
df = spark.createDataFrame([('100-200',)], ['str'])
df.select(regexp_extract('str', '(\d+)-(\d+)', 1).alias('r')).collect()

[Row(r='100')]

In [297]:
df.select(regexp_extract('str', '(\d+)-(\d+)', 2).alias('r')).collect()

[Row(r='200')]

## UDF Functions

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#pyspark.sql.functions.udf

In [299]:
# create a new UDF
slen = udf(lambda s: len(s), IntegerType())

dic = [
    {'name': 'Alice'},
    {'name': 'Bob'},
    {'name': 'Cindy'},
      ]

df = spark.createDataFrame(dic)
df.select(slen('name')).show()

+--------------+
|<lambda>(name)|
+--------------+
|             5|
|             3|
|             5|
+--------------+



## Window Functions

https://stackoverflow.com/questions/41661068/group-by-rank-and-aggregate-spark-data-frame-using-pyspark

## Close Session

In [44]:
spark.stop()